In [6]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import pandas as pd

In [7]:
results_folder = "results/"

J_list = [100] # personas
M_list = [50] # mesas
G_list = [2] # grupos
I_list = [2] # candidatos 
lambda_list = [0.5]
cv_list = [1000]
seed_list = [i+1 for i in range(20)]
# seed_list = [i+1 for i in range(20)]

instances = []
n_instances = len(J_list)*len(M_list)*len(G_list)*len(I_list)*len(seed_list)
EM_method_names = ["full"]


In [8]:

df_instances = []

# read instances
for J in J_list:
    for M in M_list:
        for G in G_list:
            for I in I_list:
                for lambda_ in lambda_list:
                    for seed in seed_list:
                        for cv in cv_list:
                            for EM_method in EM_method_names:
                                pickle_path = f'J{J}_M{M}_G{G}_I{I}_lambda{int(100*lambda_)}/{EM_method}_cv{cv}_convergence/{seed}.pickle'
                                file_path = os.path.join(results_folder, pickle_path)
                                if os.path.exists(file_path):
                                    # read pickle
                                    with open(file_path, 'rb') as handle:
                                        instance = pickle.load(handle)
                                    # print(instance)
                                    p_est = instance['p_est']
                                    df_instances.append([J,M,G,I,lambda_,seed,cv,EM_method,p_est])
                        
                                else:
                                    print("File not found:", file_path)
                                    print("File path:", file_path)
                                    print("")

In [9]:
df_instances = pd.DataFrame(df_instances, columns=['J','M','G','I','lambda','seed','cv','EM_method','p_est'])

# print all rows
pd.set_option('display.max_rows', None)
# df_instances

In [10]:
J_list = [100] # personas
M_list = [50] # mesas
G_list = [2,3] # grupos
I_list = [2,3] # candidatos 
lambda_list = [0.5]
# cv_list = [1000, 10000]
cv_list = [1000, 10000]
seed_list = [i+1 for i in range(20)]
# seed_list = [i+1 for i in range(20)]

instances = []
n_instances = len(J_list)*len(M_list)*len(G_list)*len(I_list)*len(seed_list)
EM_method_names = ["full"]

df_differences = []

# read instances
for J in J_list:
    for M in M_list:
        for G in G_list:
            for I in I_list:
                for lambda_ in lambda_list:
                    for cv in cv_list:
                        for seed1 in seed_list:
                            for seed2 in seed_list[seed1:]:
                                pickle_path1 = f'J{J}_M{M}_G{G}_I{I}_lambda{int(100*lambda_)}/full_cv{cv}_convergence/{seed1}.pickle'
                                file_path1 = os.path.join(results_folder, pickle_path1)
                                if os.path.exists(file_path1):
                                    # read pickle
                                    with open(file_path1, 'rb') as handle:
                                        instance1 = pickle.load(handle)
                                    # print(instance)
                                    p_est1 = instance1['p_est']
                                    X1 = instance1['X']
                                else:
                                    print("File not found:", file_path)
                                    print("File path:", file_path)
                                    print("")
                                # same for seed2
                                pickle_path2 = f'J{J}_M{M}_G{G}_I{I}_lambda{int(100*lambda_)}/full_cv{cv}_convergence/{seed2}.pickle'
                                file_path2 = os.path.join(results_folder, pickle_path2)
                                if os.path.exists(file_path2):
                                    # read pickle
                                    with open(file_path2, 'rb') as handle:
                                        instance2 = pickle.load(handle)
                                    # print(instance)
                                    p_est2 = instance2['p_est']
                                    X2 = instance2['X']
                                # dif_instance = np.max(np.abs(p_est1 - p_est2))
                                dif_instance = np.mean(np.abs(p_est1 - p_est2))
                                # frobenius norm
                                # dif_instance = np.linalg.norm(p_est1 - p_est2)
                                # print(seed1, seed2, dif_instance)
                                # append the dif
                                df_differences.append([J,M,G,I,lambda_,cv,seed1,seed2,dif_instance])
                                # else:
                                #     print("File not found:", file_path)
                                #     print("File path:", file_path)
                                #     print("")

df_differences = pd.DataFrame(df_differences, columns=['J','M','G','I','lambda','cv','seed1','seed2','dif_instance'])

In [11]:
# pasar a latex la tabla
df_differences_latex = df_differences[df_differences['cv'] == 1000]
df_differences_latex = df_differences[['G', 'I', 'dif_instance']]
# round to 4 decimals latex
print(df_differences_latex.groupby(['G','I']).agg(
    ['mean', 'std', 'max'])[['dif_instance']].round(4).to_latex(
        float_format="{:0.4f}".format))

\begin{tabular}{llrrr}
\toprule
 &  & \multicolumn{3}{r}{dif_instance} \\
 &  & mean & std & max \\
G & I &  &  &  \\
\midrule
\multirow[t]{2}{*}{2} & 2 & 0.0022 & 0.0021 & 0.0059 \\
 & 3 & 0.0014 & 0.0009 & 0.0036 \\
\cline{1-5}
\multirow[t]{2}{*}{3} & 2 & 0.0019 & 0.0014 & 0.0045 \\
 & 3 & 0.0015 & 0.0007 & 0.0036 \\
\cline{1-5}
\bottomrule
\end{tabular}

